# Import Libraries



In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd

### IGNORED, DISMISSED: How the NYPD Neglects 311 Complaints about Driver Misconduct

1. Go to the New Data Portal: https://opendata.cityofnewyork.us/
2. Find each dataset. 311 requests and police precinct
3. Add a data notes section to your github readme. This should include information about the source of your data, where it came from, how often it is updated, and the period of data it contains (e.g. 2010-present or 2012 to 2019)
4. Lastly, test that this import works. This may take a while as the dataset is large.

In [ ]:
# query NYC 311 for driver misconduct
# value list
complaint_types_dm = ('Blocked Bike Lane', 'Blocked Crosswalk', 'Blocked Hydrant', 'Blocked Sidewalk', 'Chronic Speeding', 'Chronic Stoplight Violation','Detached Trailer', 'Double Parked Blocking Traffic',
       'Double Parked Blocking Vehicle', 'Drag Racing',
       'Overnight Commercial Storage', 'Parking Permit Improper Use',
       'Posted Parking Sign Violation', 'Truck Route Violation',
       'Unauthorized Bus Layover', 'Abandoned Vehicle With License Plate' )
query = "https://data.cityofnewyork.us/resource/erm2-nwe9.csv?$query=" +  requests.utils.quote(f"SELECT * WHERE descriptor IN {complaint_types_dm}  LIMIT 3000000")
dm  = pd.read_csv(query)

In [ ]:
# get police precint data

police = gpd.read_file('https://data.cityofnewyork.us/api/geospatial/78dh-3ptz?method=export&format=GeoJSON')

#Inspecting the Data
* .head()
* .tail()
* .columns
* .info()
* .shape

In [ ]:
dm.head(5)

In [ ]:
police.head(5)

In [ ]:
dm.tail(5)

In [ ]:
police.tail(5)

In [ ]:
dm.columns

In [ ]:
police.columns

In [ ]:
dm.info()

In [ ]:
police.info()

In [ ]:
dm.shape

In [ ]:
police.shape

#Creating Column "Duration"
* a column based on two other columns

In [ ]:
dm['created_date'] = pd.to_datetime(dm['created_date'])
dm['closed_date'] = pd.to_datetime(dm['closed_date'])
dm['duration'] = dm['closed_date'] - dm['created_date']

#Filter
Response times under 5 minutes
* pd.Timedelta()

    (one method we didn’t learn in class by reviewing documentation
from a Python library)

In [ ]:
dm_filtered = dm[dm['duration'] <= pd.Timedelta(minutes=5)]
dm.sample(5)

#Add Column

In [ ]:
data = dm_filtered['created_date'].value_counts()

In [ ]:
def get_day_of_week(date_str):
    date_obj = pd.to_datetime(date_str)
    return date_obj.day_name()


dm_filtered.loc[:, 'day_of_week'] = dm_filtered['created_date'].apply(get_day_of_week)


day_counts = dm_filtered.groupby('day_of_week').size().reset_index(name='day_count')


dm_filtered.sample()

#Loop and Conditional Statement

In [ ]:
for index, row in dm_filtered.iterrows():
    duration = pd.to_timedelta(row['duration'])
    if duration.seconds < 60:
        duration = str(duration.seconds) + " seconds"
    else:
        minutes = duration.seconds // 60
        seconds = duration.seconds % 60
        duration = str(minutes) + " minutes and " + str(seconds) + " seconds"
    dm_filtered.at[index, 'duration'] = duration